In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


In [2]:
csv_read = pd.read_csv("../city_data.csv")

In [3]:
city_df = pd.DataFrame(csv_read)
city_df.head()

,Unnamed: 0,City,Latitude,Longitude,Max_temp,Humidity,Clouds,Wind_speed,Country,Date
0,0,kapaa,22.0752,-159.3190,73.0,88,40,9.22,US,1612392712
1,1,yellowknife,62.4560,-114.3525,-15.0,69,90,6.91,CA,1612392485
2,2,clinton,42.5870,-82.9199,37.4,51,20,5.75,US,1612392517
3,3,hobart,-42.8794,147.3294,61.0,94,75,10.36,AU,1612392712
4,4,vaini,-21.2000,-175.2000,82.4,89,75,10.36,TO,1612392712


In [5]:
gmaps.configure(api_key=g_key)

humidity = city_df["Humidity"].astype(float)
locations = city_df[["Latitude", "Longitude"]]
fig = gmaps.figure(zoom_level=2, center = (15,20))

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=3)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))

In [30]:
# perfect weather conditions
per_max_temp = []
per_humidity = []
per_wind = []
per_cloud = []
city_list = []
lat = []
lng = []
country = []

for index, city in city_df.iterrows():
    if city_df.loc[index, "Max_temp"] >= 70 and city_df.loc[index, "Max_temp"] <= 90:
        if city_df.loc[index, 'Humidity'] < 50:
            if city_df.loc[index, "Wind_speed"] < 15:
                if city_df.loc[index, "Clouds"] <= 35:
                    city_list.append(city_df.loc[index, "City"])
                    lat.append(city_df.loc[index, "Latitude"])
                    lng.append(city_df.loc[index, "Longitude"])
                    per_max_temp.append(city_df.loc[index, "Max_temp"])
                    per_humidity.append(city_df.loc[index, "Humidity"])
                    per_wind.append(city_df.loc[index, "Wind_speed"])
                    per_cloud.append(city_df.loc[index, "Clouds"])
                    country.append(city_df.loc[index, "Country"])
                    
perfect_city = {"City": city_list,
                "Country": country,
               "Latitude": lat, 
               "Longitude": lng,
               "Max_temp": per_max_temp, 
               "Humidity": per_humidity, 
               "Wind_speed": per_wind, 
               "Clouds": per_cloud}

perfect_city_df = pd.DataFrame(perfect_city)
perfect_city_df

,City,Country,Latitude,Longitude,Max_temp,Humidity,Wind_speed,Clouds
0,miraflores,MX,19.2167,-98.8083,77.00,8,8.05,1
1,tchaourou,BJ,8.8865,2.5975,77.90,18,5.14,0
2,abeche,TD,13.8292,20.8324,73.08,19,12.66,29
3,tame,CO,6.4610,-71.7300,84.07,44,4.74,16
4,taoudenni,ML,22.6783,-3.9836,71.22,21,9.82,21
5,kita,ML,13.0349,-9.4895,74.77,16,3.53,34
6,cabo san lucas,MX,22.8909,-109.9124,84.20,30,8.05,20
7,ouallam,NE,14.3188,2.0866,74.03,16,9.40,0
8,hilo,US,19.7297,-155.0900,84.20,39,11.50,1
9,vallenar,CL,-28.5708,-70.7581,80.29,31,5.21,0


In [40]:
hotel_lat = []
hotel_lng = []
hotel_name = []
hotel_city = []
hotel_country = []

for index, city in perfect_city_df.iterrows():
    target_coordinates = f"{perfect_city_df.loc[index, 'Latitude']},{perfect_city_df.loc[index, 'Longitude']}"
    target_radius = 5000 #meters
    target_type = "hotel"
    

    # set up a parameters dictionary
    params = {
        "location": target_coordinates,
        "keyword": "hotel",
        "radius": target_radius,
        "type": target_type,
        "key": g_key
    }

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# run a request using our params dictionary
    response = requests.get(base_url, params=params)
    hotel_data = response.json()
    try:
        hotel_name.append(hotel_data["results"][0]["name"].title())
        hotel_lat.append(hotel_data["results"][0]["geometry"]["location"]["lat"])
        hotel_lng.append(hotel_data["results"][0]["geometry"]["location"]["lng"])
        hotel_city.append(perfect_city_df.loc[index, "City"].title())
        hotel_country.append(perfect_city_df.loc[index,"Country"])
    except IndexError:
        print("hotel not found")
    



hotel not found
hotel not found
hotel not found


In [41]:
hotel_name

['Hotel Mansión',
 'Safir Hotel',
 'Hôtel Kinpeski',
 'Royal Bacarly Hotel',
 'Hôtel Le Relais',
 'Waldorf Astoria Los Cabos Pedregal',
 'Hilo Hawaiian Hotel',
 'Hotel Solaris',
 'Alwatane',
 'Custodian Hotel Gombe',
 'Kuleu Hotel',
 'Hotel Austral Viedma',
 "Ka'Ana Resort",
 'Jega Guest Inn']

In [42]:
hotel_info = {"Name": hotel_name,
              "City": hotel_city,
              "Country": hotel_country,
              "Latitude": hotel_lat, 
              "Longitude": hotel_lng}
hotel_info_df = pd.DataFrame(hotel_info)
hotel_info_df

,Name,City,Country,Latitude,Longitude
0,Hotel Mansión,Miraflores,MX,19.195321,-98.801086
1,Safir Hotel,Tchaourou,BJ,8.887971,2.594124
2,Hôtel Kinpeski,Abeche,TD,13.820921,20.792320
3,Royal Bacarly Hotel,Tame,CO,6.459659,-71.734117
4,Hôtel Le Relais,Kita,ML,13.039194,-9.487297
5,Waldorf Astoria Los Cabos Pedregal,Cabo San Lucas,MX,22.874811,-109.912136
6,Hilo Hawaiian Hotel,Hilo,US,19.728404,-155.066467
7,Hotel Solaris,Vallenar,CL,-28.572451,-70.762866
8,Alwatane,Massaguet,TD,12.481157,15.438398
9,Custodian Hotel Gombe,Gombe,NG,10.283781,11.148841


In [ ]:
markers = 